<a href="https://colab.research.google.com/github/univer-coder/Deep-learning-course/blob/main/lecture03_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第3回講義 宿題

## 課題

今回のLessonで学んだことを元に，MNISTのファッション版 (Fashion MNIST，クラス数10) を多層パーセプトロンによって分類してみましょう．

Fashion MNISTの詳細については以下のリンクを参考にしてください．

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値

Accuracy 85%

### ルール

- 訓練データは`x_train`， `t_train`，テストデータは`x_test`で与えられます．
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください．
- **下のセルで指定されている`x_train`，`t_train`以外の学習データは使わないでください．**
- **多層パーセプトロンのアルゴリズム部分は第3回の演習を参考に，NumPyのみで実装してください．** (sklearnやtensorflowなどは使用しないでください)．
    - データの前処理部分でsklearnの関数を使う (例えば `sklearn.model_selection.train_test_split`) のは問題ありません．

### 提出方法
- 2つのファイルを提出していただきます．
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し，**Omnicampusの宿題タブから「第3回 ニューラルネットワーク基礎」を選択して**提出してください．
    2. それに対応するpythonのコードを`submission_code.py`として保存し，**Omnicampusの宿題タブから「第3回 ニューラルネットワーク基礎 (code)」を選択して**提出してください．pythonファイル自体の提出ではなく，「提出内容」の部分にコードをコピー&ペーストしてください．
      
- なお，採点は1で行い，2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）．コードの内容を変更した場合は，**1と2の両方を提出し直してください**．

### 評価方法
- 予測ラベルの`t_test`に対する精度 (Accuracy) で評価します．
- 即時採点しLeader Boardを更新します（採点スケジュールは別アナウンス）．
- 締切時の点数を最終的な評価とします．

### ドライブのマウント

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### データの読み込み（このセルは修正しないでください）

In [18]:
import os
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import inspect


#学習データ
x_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture03/data/x_train.npy')
t_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture03/data/y_train.npy')

#テストデータ
x_test = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture03/data/x_test.npy')

# データの前処理（正規化， one-hot encoding)
x_train, x_test = x_train / 255., x_test / 255.
x_train, x_test = x_train.reshape(x_train.shape[0], -1), x_test.reshape(x_test.shape[0], -1)
t_train = np.eye(N=10)[t_train.astype("int32").flatten()]

### 多層パーセプトロンの実装

In [19]:
# データの分割
x_train, x_val, t_train, t_val =\
    train_test_split(x_train, t_train, test_size=10000)

In [20]:
def np_log(x):
    return np.log(np.clip(x, 1e-10, 1e+10))


def create_batch(data, batch_size):
    """
    :param data: np.ndarray，入力データ
    :param batch_size: int，バッチサイズ
    """
    num_batches, mod = divmod(data.shape[0], batch_size)
    batched_data = np.split(data[: batch_size * num_batches], num_batches)
    if mod:
        batched_data.append(data[batch_size * num_batches:])

    return batched_data

In [25]:
# シード値を変えることで何が起きるかも確かめてみてください．
rng = np.random.RandomState(1234)
random_state = 42


# 発展: 今回の講義で扱っていない活性化関数について調べ，実装してみましょう
def relu(x):
    return np.maximum(x, 0)


def deriv_relu(x):
    return (x > 0).astype(x.dtype)


def gelu(x):
    return 0.5 * x * (1 + np.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * np.power(x, 3))))


def deriv_gelu(x):
    tanh_term = np.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * x**3))
    sech2_term = 1 - tanh_term**2
    coeff = np.sqrt(2 / np.pi)
    inner = x + 0.044715 * x**3
    grad = 0.5 * (1 + tanh_term + x * sech2_term * coeff * (1 + 3 * 0.044715 * x**2))
    return grad


def softmax(x):
    x -= x.max(axis=1, keepdims=True)
    x_exp = np.exp(x)
    return x_exp / np.sum(x_exp, axis=1, keepdims=True)


def deriv_softmax(x):
    return softmax(x) * (1 - softmax(x))


def crossentropy_loss(t, y):
    return (- t * np_log(y) - (1 - t) * np_log(1 - y)).mean()

# 過学習を防ぐdropout
def dropout(x, drop_rate):
    mask = np.random.binomial(1, 1 - drop_rate, size=x.shape)
    return x * mask / (1 - drop_rate)


class Dense:
    def __init__(self, in_dim, out_dim, function, deriv_function):
        self.W = np.random.uniform(low=-0.08, high=0.08,
                                   size=(in_dim, out_dim)).astype("float64")
        self.b = np.zeros(out_dim).astype("float64")
        self.function = function
        self.deriv_function = deriv_function

        self.x = None
        self.u = None

        self.dW = None
        self.db = None

        self.params_idxs = np.cumsum([self.W.size, self.b.size])

    def __call__(self, x):
        """
        順伝播処理を行うメソッド．
        x: (batch_size, in_dim_{j})
        h: (batch_size, out_dim_{j})
        """
        self.x = x
        self.u = self.x @ self.W + self.b
        h = self.function(self.u) #functionは活性化関数. 非線形化
        return h

    def b_prop(self, delta, W):
        """
        誤差逆伝播を行うメソッド．
        delta (=delta_{j+1}): (batch_size, out_dim_{j+1})
        W (=W_{j+1}): (out_dim_{j}, out_dim_{j+1})
        self.delta (=delta_{j}): (batch_size, out_dim_{j})
        """
        self.delta = self.deriv_function(self.u) * (delta @ W.T)
        return self.delta

    def compute_grad(self):
        """
        勾配を計算するメソッド．
        self.x: (batch_size, in_dim_{j})
        self.delta: (batch_size, out_dim_{j})
        self.dW: (in_dim_{j}, out_dim_{j})
        self.db: (out_dim_{j})
        """
        batch_size = self.delta.shape[0]

        self.dW = self.x.T @ self.delta / batch_size
        self.db = np.ones(batch_size) @ self.delta / batch_size

    def get_params(self):
        return np.concatenate([self.W.ravel(), self.b], axis=0)

    def set_params(self, params):
        """
        params: List[np.ndarray, np.ndarray]
            1つ目の要素が重みW: (in_dim, out_dim)，2つ目の要素がバイアス: (out_dim,)
        """
        _W, _b = np.split(params, self.params_idxs)[:-1]
        self.W = _W.reshape(self.W.shape)
        self.b = _b

    def get_grads(self):
        return np.concatenate([self.dW.ravel(), self.db], axis=0)


class Model:
    def __init__(self, hidden_dims, activation_functions, deriv_functions):
        """
        :param hiden_dims: List[int]，各層のノード数を格納したリスト．
        :params activation_functions: List, 各層で用いる活性化関数を格納したリスト．
        :params derive_functions: List，各層で用いる活性化関数の導関数を格納したリスト．
        """
        # 各層をリストに格納していく
        self.layers = []
        for i in range(len(hidden_dims)-2):  # 出力層以外は同じ構造
            self.layers.append(Dense(hidden_dims[i], hidden_dims[i+1],
                                     activation_functions[i], deriv_functions[i]))
        self.layers.append(Dense(hidden_dims[-2], hidden_dims[-1],
                                 activation_functions[-1], deriv_functions[-1]))  # 出力層を追加

    def __call__(self, x, drop_rate, train=True):
        return self.forward(x, drop_rate, train)

    def forward(self, x, drop_rate, train=True):
        """順伝播処理を行うメソッド"""
        for i, layer in enumerate(self.layers):
            x = layer(x)
            # 最後の出力層（softmax）には dropout をかけない
            if train and i < len(self.layers) - 1:
                x = dropout(x, drop_rate)
        return x



    def backward(self, delta):
        """誤差逆伝播，勾配計算を行うメソッド"""
        batch_size = delta.shape[0]

        for i, layer in enumerate(self.layers[::-1]):
            if i == 0:  # 出力層の場合
                layer.delta = delta  # y - t, modelを訓練するtrain_mst関数で定義される
                layer.compute_grad() # 勾配の計算
            else:  # 出力層以外の場合
                delta = layer.b_prop(delta, W)  # 逆伝播 Denseクラス内の関数で定義
                layer.compute_grad()  # 勾配の計算

            W = layer.W

    def update(self, eps):
        """パラメータの更新を行うメソッド"""
        for layer in self.layers:
            layer.W -= eps * layer.dW
            layer.b -= eps * layer.db

drop_rate = 0.05
n_epochs = 50
batch_size = 64
mlp = Model(hidden_dims=[784, 512, 216, 128, 64, 32, 10],
              activation_functions=[relu, gelu, relu, relu, relu, relu, softmax],
              deriv_functions=[deriv_relu, deriv_gelu, deriv_relu, deriv_relu, deriv_relu, deriv_relu, deriv_softmax])

### モデルの学習

In [26]:
def train_model(mlp, x_train, t_train, x_val, t_val, n_epochs, patience=10):
    best_val_accuracy = 0
    best_weights = None
    patience_counter = 0

    for epoch in range(n_epochs):
        losses_train = []
        losses_valid = []
        train_num = 0
        train_true_num = 0
        valid_num = 0
        valid_true_num = 0

        x_train, t_train = shuffle(x_train, t_train)
        x_train_batches, t_train_batches = create_batch(x_train, batch_size), create_batch(t_train, batch_size)

        x_val, t_val = shuffle(x_val, t_val)
        x_val_batches, t_val_batches = create_batch(x_val, batch_size), create_batch(t_val, batch_size)

        lr = 0.1 * (0.97 ** (epoch // 5))

        # モデルの訓練
        for x, t in zip(x_train_batches, t_train_batches):
            # 順伝播
            y = mlp(x, drop_rate, train=True)

            # 損失の計算
            loss = crossentropy_loss(t, y)
            losses_train.append(loss.tolist())

            # 逆伝播
            delta = y - t
            mlp.backward(delta)

            # パラメータの更新
            mlp.update(lr)

            # 精度を計算
            acc = accuracy_score(t.argmax(axis=1), y.argmax(axis=1), normalize=False)
            train_num += x.shape[0]
            train_true_num += acc

        # モデルの評価
        for x, t in zip(x_val_batches, t_val_batches):
            # 順伝播
            y = mlp(x, drop_rate, train=False)

            # 損失の計算
            loss = crossentropy_loss(t, y)
            losses_valid.append(loss.tolist())

            acc = accuracy_score(t.argmax(axis=1), y.argmax(axis=1), normalize=False)
            valid_num += x.shape[0]
            valid_true_num += acc

        print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
            epoch,
            np.mean(losses_train),
            train_true_num/train_num,
            np.mean(losses_valid),
            valid_true_num/valid_num
        ))

        # Early stopping 条件のチェック
        current_val_accuracy = valid_true_num / valid_num
        if current_val_accuracy > best_val_accuracy:
            best_val_accuracy = current_val_accuracy
            best_weights = [layer.W.copy() for layer in mlp.layers]  # 重みを保存（deep copy）
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping triggered at epoch {epoch}")
            break

    # 一番良かった重みに戻す
    if best_weights:
        for layer, best_W in zip(mlp.layers, best_weights):
            layer.W = best_W


train_model(mlp, x_train, t_train, x_val, t_val, n_epochs)

EPOCH: 0, Train [Loss: 0.210, Accuracy: 0.464], Valid [Loss: 0.132, Accuracy: 0.703]
EPOCH: 1, Train [Loss: 0.113, Accuracy: 0.749], Valid [Loss: 0.119, Accuracy: 0.720]
EPOCH: 2, Train [Loss: 0.095, Accuracy: 0.795], Valid [Loss: 0.108, Accuracy: 0.765]
EPOCH: 3, Train [Loss: 0.085, Accuracy: 0.823], Valid [Loss: 0.284, Accuracy: 0.612]
EPOCH: 4, Train [Loss: 0.078, Accuracy: 0.840], Valid [Loss: 0.078, Accuracy: 0.840]
EPOCH: 5, Train [Loss: 0.073, Accuracy: 0.850], Valid [Loss: 0.098, Accuracy: 0.807]
EPOCH: 6, Train [Loss: 0.070, Accuracy: 0.856], Valid [Loss: 0.071, Accuracy: 0.856]
EPOCH: 7, Train [Loss: 0.068, Accuracy: 0.862], Valid [Loss: 0.076, Accuracy: 0.850]
EPOCH: 8, Train [Loss: 0.066, Accuracy: 0.866], Valid [Loss: 0.067, Accuracy: 0.864]
EPOCH: 9, Train [Loss: 0.063, Accuracy: 0.872], Valid [Loss: 0.063, Accuracy: 0.871]
EPOCH: 10, Train [Loss: 0.062, Accuracy: 0.875], Valid [Loss: 0.114, Accuracy: 0.798]
EPOCH: 11, Train [Loss: 0.060, Accuracy: 0.879], Valid [Loss: 0.

In [23]:
t_pred = []
for x in x_test:
    # 順伝播
    x = x[np.newaxis, :]
    y = mlp(x, drop_rate, train=False)

    # モデルの出力を予測値のスカラーに変換
    pred = y.argmax(1).tolist()

    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture03/submission_pred10.csv', header=True, index_label='id')